## Part B: Predictive Modelling

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/zomato_df_final_data.csv')
display(df.head())
print('\nDataset dimensions:', df.shape)

,address,cost,cuisine,lat,link,lng,phone,rating_number,rating_text,subzone,title,type,votes,groupon,color,cost_2,cuisine_color
0,"371A Pitt Street, CBD, Sydney",50.0,"['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']",-33.876059,https://www.zomato.com/sydney/sydney-madang-cbd,151.207605,02 8318 0406,4.0,Very Good,CBD,Sydney Madang,['Casual Dining'],1311.0,False,#e15307,5.243902,#6f706b
1,"Shop 7A, 2 Huntley Street, Alexandria, Sydney",80.0,"['Cafe', 'Coffee and Tea', 'Salad', 'Poké']",-33.910999,https://www.zomato.com/sydney/the-grounds-of-a...,151.193793,02 9699 2225,4.6,Excellent,"The Grounds of Alexandria, Alexandria",The Grounds of Alexandria Cafe,['Café'],3236.0,False,#9c3203,7.560976,#6f706b
2,"Level G, The Darling at the Star, 80 Pyrmont ...",120.0,['Japanese'],-33.867971,https://www.zomato.com/sydney/sokyo-pyrmont,151.195210,1800 700 700,4.9,Excellent,"The Star, Pyrmont",Sokyo,['Fine Dining'],1227.0,False,#7f2704,10.650407,#6f706b
3,"Sydney Opera House, Bennelong Point, Circular...",270.0,['Modern Australian'],-33.856784,https://www.zomato.com/sydney/bennelong-restau...,151.215297,02 9240 8000,4.9,Excellent,Circular Quay,Bennelong Restaurant,"['Fine Dining', 'Bar']",278.0,False,#7f2704,22.235772,#4186f4
4,"20 Campbell Street, Chinatown, Sydney",55.0,"['Thai', 'Salad']",-33.879035,https://www.zomato.com/sydney/chat-thai-chinatown,151.206409,02 8317 4811,4.5,Excellent,Chinatown,Chat Thai,['Casual Dining'],2150.0,False,#a83703,5.630081,#6f706b



Dataset dimensions: (10500, 17)


In [3]:
df.describe()

,cost,lat,lng,rating_number,votes,cost_2
count,10154.000000,10308.000000,10308.000000,7184.000000,7184.000000,10154.000000
mean,51.153240,-32.921377,148.067359,3.283672,83.581013,5.332974
std,27.799485,8.263449,26.695402,0.454580,175.117966,2.147115
min,8.000000,-37.858473,-123.270371,1.800000,4.000000,2.000000
25%,30.000000,-33.899094,151.061061,3.000000,12.000000,3.699187
50%,45.000000,-33.872741,151.172468,3.300000,32.000000,4.857724
75%,60.000000,-33.813451,151.208940,3.600000,87.000000,6.016260
max,500.000000,51.500986,152.869052,4.900000,3236.000000,40.000000


In [17]:
df.dtypes

address           object
cost             float64
cuisine           object
lat              float64
link              object
lng              float64
phone             object
rating_number    float64
rating_text       object
subzone           object
title             object
type              object
votes            float64
groupon             bool
color             object
cost_2           float64
cuisine_color     object
dtype: object

### 1. Feature Engineering

#### 1.1. NA Handling

<p>
    We can see missing values make up 31.6% of the total number for rows for rating_number, rating_text, and votes. Considering that rating_number and rating_text are our target variables, we cannot reliable impute them. Therefore, we shall remove all of these rows.
</p>

In [4]:
na_summary_table = pd.DataFrame({
    'Missing Values': df.isna().sum(),
    'Percentage': df.isna().mean()
})
print(na_summary_table)

               Missing Values  Percentage
address                     0    0.000000
cost                      346    0.032952
cuisine                     0    0.000000
lat                       192    0.018286
link                        0    0.000000
lng                       192    0.018286
phone                       0    0.000000
rating_number            3316    0.315810
rating_text              3316    0.315810
subzone                     0    0.000000
title                       0    0.000000
type                       48    0.004571
votes                    3316    0.315810
groupon                     0    0.000000
color                       0    0.000000
cost_2                    346    0.032952
cuisine_color               0    0.000000


In [5]:
df_clean = df.dropna(subset=['rating_number']) # Create subset of the dataset where rating_number is not NA

<p>
We imputed missing values in the cost column using the median of the available data, as it is robust to outliers and better reflects the skewed distribution of restaurant prices. This ensured the column was complete without distorting the overall distribution.
</p>

In [6]:
na_summary_table = pd.DataFrame({
    'Missing Values': df_clean.isna().sum(),
    'Percentage': df_clean.isna().mean()
})
print(na_summary_table)

               Missing Values  Percentage
address                     0    0.000000
cost                      103    0.014337
cuisine                     0    0.000000
lat                       115    0.016008
link                        0    0.000000
lng                       115    0.016008
phone                       0    0.000000
rating_number               0    0.000000
rating_text                 0    0.000000
subzone                     0    0.000000
title                       0    0.000000
type                       21    0.002923
votes                       0    0.000000
groupon                     0    0.000000
color                       0    0.000000
cost_2                    103    0.014337
cuisine_color               0    0.000000


In [7]:
# List of numeric columns to impute
cols_to_impute = ['cost', 'cost_2']   # add more columns here if needed

for col in cols_to_impute:
    median_value = df_clean[col].median()
    df_clean.loc[df_clean[col].isna(), col] = median_value

In [8]:
na_summary_table = pd.DataFrame({
    'Missing Values': df_clean.isna().sum(),
    'Percentage': df_clean.isna().mean()
})
print(na_summary_table)

               Missing Values  Percentage
address                     0    0.000000
cost                        0    0.000000
cuisine                     0    0.000000
lat                       115    0.016008
link                        0    0.000000
lng                       115    0.016008
phone                       0    0.000000
rating_number               0    0.000000
rating_text                 0    0.000000
subzone                     0    0.000000
title                       0    0.000000
type                       21    0.002923
votes                       0    0.000000
groupon                     0    0.000000
color                       0    0.000000
cost_2                      0    0.000000
cuisine_color               0    0.000000


<p>
Missing values in the type column were imputed with the placeholder "Unknown" to preserve all rows and avoid biasing the dataset. This approach ensures transparency while maintaining consistency across categorical values.
</p>

In [9]:
df_clean.loc[df_clean['type'].isna(), 'type'] = [['Unknown']]

<p>
Missing latitude and longitude values were imputed using the median coordinates of their respective subzone, ensuring restaurants could still be included in geospatial analysis. This approach preserves data completeness while maintaining spatial consistency at the suburb level. However, we are still left with 14 missing values in both latitude and longitude.
</p>

In [10]:
# Impute latitude by suburb
df_clean.loc[df_clean['lat'].isna(), 'lat'] = (
    df_clean.groupby('subzone')['lat'].transform('median')
)

# Impute longitude by suburb
df_clean.loc[df_clean['lng'].isna(), 'lng'] = (
    df_clean.groupby('subzone')['lng'].transform('median')
)

In [11]:
na_summary_table = pd.DataFrame({
    'Missing Values': df_clean.isna().sum(),
    'Percentage': df_clean.isna().mean()
})
print(na_summary_table)

               Missing Values  Percentage
address                     0    0.000000
cost                        0    0.000000
cuisine                     0    0.000000
lat                        14    0.001949
link                        0    0.000000
lng                        14    0.001949
phone                       0    0.000000
rating_number               0    0.000000
rating_text                 0    0.000000
subzone                     0    0.000000
title                       0    0.000000
type                        0    0.000000
votes                       0    0.000000
groupon                     0    0.000000
color                       0    0.000000
cost_2                      0    0.000000
cuisine_color               0    0.000000


<p>
For the remaining 14 rows, we standardized the subzone by extracting the area name after the final comma (e.g., Sydney, CBD → CBD) and then imputed missing coordinates using the median latitude and longitude of this cleaned subzone. This refinement allowed us to recover additional locations while maintaining consistency with the broader geospatial imputation strategy.
</p>

In [12]:
# Create subzone_clean safely with .loc
df_clean.loc[:, 'subzone_clean'] = (
    df_clean['subzone'].str.split(',').str[-1].str.strip()
)

# Impute latitude
df_clean.loc[df_clean['lat'].isna(), 'lat'] = (
    df_clean.groupby('subzone_clean')['lat'].transform('median')
)

# Impute longitude
df_clean.loc[df_clean['lng'].isna(), 'lng'] = (
    df_clean.groupby('subzone_clean')['lng'].transform('median')
)

C:\Users\Nigel\AppData\Local\Temp\ipykernel_668\2552737669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'subzone_clean'] = (


In [16]:
na_summary_table = pd.DataFrame({
    'Missing Values': df_clean.isna().sum(),
    'Percentage': df_clean.isna().mean()
})
print(na_summary_table)

               Missing Values  Percentage
address                     0         0.0
cost                        0         0.0
cuisine                     0         0.0
lat                         0         0.0
link                        0         0.0
lng                         0         0.0
phone                       0         0.0
rating_number               0         0.0
rating_text                 0         0.0
subzone                     0         0.0
title                       0         0.0
type                        0         0.0
votes                       0         0.0
groupon                     0         0.0
color                       0         0.0
cost_2                      0         0.0
cuisine_color               0         0.0
subzone_clean               0         0.0


#### 1.3. Create Useful Features

In [18]:
# Create cuisine diversity feature
df_clean.loc[:, 'cuisine_diversity'] = df_clean['cuisine'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Quick check
print(df_clean[['cuisine', 'cuisine_diversity']].head())
print(df_clean['cuisine_diversity'].describe())

                                       cuisine  cuisine_diversity
0   ['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']                  0
1  ['Cafe', 'Coffee and Tea', 'Salad', 'Poké']                  0
2                                 ['Japanese']                  0
3                        ['Modern Australian']                  0
4                            ['Thai', 'Salad']                  0
count    7184.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: cuisine_diversity, dtype: float64


C:\Users\Nigel\AppData\Local\Temp\ipykernel_668\3701837201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'cuisine_diversity'] = df_clean['cuisine'].apply(lambda x: len(x) if isinstance(x, list) else 0)
